In [1]:
import pandas as pd

train = pd.read_csv("/content/train_1.csv")
test = pd.read_csv("/content/test set.csv")
submission = pd.read_csv("/content/SubmissionFormat.csv")

In [2]:
train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


In [3]:
train.shape

(59400, 41)

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59400 entries, 0 to 59399
Data columns (total 41 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     59400 non-null  int64  
 1   amount_tsh             59400 non-null  float64
 2   date_recorded          59400 non-null  object 
 3   funder                 55765 non-null  object 
 4   gps_height             59400 non-null  int64  
 5   installer              55745 non-null  object 
 6   longitude              59400 non-null  float64
 7   latitude               59400 non-null  float64
 8   wpt_name               59400 non-null  object 
 9   num_private            59400 non-null  int64  
 10  basin                  59400 non-null  object 
 11  subvillage             59029 non-null  object 
 12  region                 59400 non-null  object 
 13  region_code            59400 non-null  int64  
 14  district_code          59400 non-null  int64  
 15  lg

In [5]:
train['status_group'].value_counts()

functional                 32259
non functional             22824
functional needs repair     4317
Name: status_group, dtype: int64

In [6]:
for i in range(len(train)):
  if train['public_meeting'][i] == True:
    train['public_meeting'][i] = 1
  elif train['public_meeting'][i] == False:
    train['public_meeting'][i] = 0
  else:
    train['public_meeting'][i] = -1

  if train['permit'][i] == True:
    train['permit'][i] = 1
  elif train['permit'][i] == False:
    train['permit'][i] = 0
  else:
    train['permit'][i] = -1


train.fillna('0', inplace=True)

from sklearn.preprocessing import OrdinalEncoder
ord_enc = OrdinalEncoder()
for col in train.columns:
    if train[col].dtype == 'object':
      print(col, end = " ")
      train[col] = ord_enc.fit_transform(train[[col]])
      print(train[col].dtype)


for col in train.columns:
  if col!='status_group':
    train[col] = train[col].astype('int')

<ipython-input-6-e75a324ba786>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['public_meeting'][i] = 1
<ipython-input-6-e75a324ba786>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['permit'][i] = 0
<ipython-input-6-e75a324ba786>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['public_meeting'][i] = -1
<ipython-input-6-e75a324ba786>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

date_recorded float64
funder float64
installer float64
wpt_name float64
basin float64
subvillage float64
region float64
lga float64
ward float64
recorded_by float64
scheme_management float64
scheme_name float64
extraction_type float64
extraction_type_group float64
extraction_type_class float64
management float64
management_group float64
payment float64
payment_type float64
water_quality float64
quality_group float64
quantity float64
quantity_group float64
source float64
source_type float64
source_class float64
waterpoint_type float64
waterpoint_type_group float64
status_group float64


In [7]:
X = train.drop(['status_group'], axis = 1)
y = train['status_group']

In [8]:
!pip install numba --upgrade
!pip install pycaret
from pycaret.classification import setup, compare_models, evaluate_model, predict_model, save_model, load_model

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numba-0.56.4-cp38-cp38-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (3.5 MB)
  Using cached llvmlite-0.39.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.6 MB)
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.37.0
    Uninstalling llvmlite-0.37.0:
      Successfully uninstalled llvmlite-0.37.0
  Attempting uninstall: numba
    Found existing installation: numba 0.54.1
    Uninstalling numba-0.54.1:
      Successfully uninstalled numba-0.54.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pycaret 2.3.10 requires numba<0.55, but you have numba 0.56.4 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached numba-0.54.1-cp38-cp3

In [9]:
s = setup(train, target='status_group')

,Description,Value
0,session_id,7086
1,Target,status_group
2,Target Type,Multiclass
3,Label Encoded,None
4,Original Data,"(59400, 41)"
5,Missing Values,False
6,Numeric Features,16
7,Categorical Features,24
8,Ordinal Features,False
9,High Cardinality Features,False


INFO:logs:create_model_container: 0
INFO:logs:master_model_container: 0
INFO:logs:display_container: 1
INFO:logs:Pipeline(memory=None,
         steps=[('dtypes',
                 DataTypes_Auto_infer(categorical_features=[],
                                      display_types=True, features_todrop=[],
                                      id_columns=[],
                                      ml_usecase='classification',
                                      numerical_features=[],
                                      target='status_group',
                                      time_features=[])),
                ('imputer',
                 Simple_Imputer(categorical_strategy='not_available',
                                fill_value_categorical=None,
                                fill_value_numerical=None,
                                numeric...
                ('scaling', 'passthrough'), ('P_transform', 'passthrough'),
                ('binn', 'passthrough'), ('rem_outliers', 'p

In [10]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.8080,0.9064,0.6647,0.8019,0.8005,0.6389,0.6428,10.260
et,Extra Trees Classifier,0.7952,0.8971,0.6679,0.7892,0.7905,0.6197,0.6213,14.167
lightgbm,Light Gradient Boosting Machine,0.7891,0.8946,0.6211,0.7896,0.7759,0.5941,0.6061,2.418
gbc,Gradient Boosting Classifier,0.7560,0.8610,0.5605,0.7622,0.7338,0.5204,0.5429,58.054
ridge,Ridge Classifier,0.7401,0.0000,0.5243,0.7371,0.7109,0.4854,0.5087,0.147
lda,Linear Discriminant Analysis,0.7372,0.8281,0.5626,0.7360,0.7206,0.4898,0.5084,2.069
dt,Decision Tree Classifier,0.7320,0.7632,0.6240,0.7337,0.7327,0.5159,0.5160,1.209
ada,Ada Boost Classifier,0.7280,0.8156,0.5274,0.7188,0.7034,0.4663,0.4837,4.134
lr,Logistic Regression,0.5570,0.6030,0.3670,0.5033,0.5020,0.0925,0.1074,35.894
dummy,Dummy Classifier,0.5444,0.5000,0.3333,0.2964,0.3838,0.0000,0.0000,0.052


INFO:logs:create_model_container: 14
INFO:logs:master_model_container: 14
INFO:logs:display_container: 2
INFO:logs:RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=7086, verbose=0,
                       warm_start=False)
INFO:logs:compare_models() succesfully completed......................................


In [11]:
evaluate_model(best_model)

INFO:logs:Initializing evaluate_model()
INFO:logs:evaluate_model(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=7086, verbose=0,
                       warm_start=False), fold=None, fit_kwargs=None, plot_kwargs=None, feature_name=None, groups=None, use_train_data=False)


interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [12]:
for i in range(len(test)):
  if test['public_meeting'][i] == True:
    test['public_meeting'][i] = 1
  elif test['public_meeting'][i] == False:
    test['public_meeting'][i] = 0
  else:
    test['public_meeting'][i] = -1

  if test['permit'][i] == True:
    test['permit'][i] = 1
  elif test['permit'][i] == False:
    test['permit'][i] = 0
  else:
    test['permit'][i] = -1

test.fillna('0', inplace=True)

for col in test.columns:
  if test[col].dtype == 'object':
    print(col, end = " ")
    test[col] = ord_enc.fit_transform(test[[col]])
    print(test[col].dtype)

for col in test.columns:
  test[col] = test[col].astype('int')

date_recorded float64
funder float64
installer float64
wpt_name float64
basin float64
subvillage float64
region float64
lga float64
ward float64
recorded_by float64
scheme_management float64
scheme_name float64
extraction_type float64
extraction_type_group float64
extraction_type_class float64
management float64
management_group float64
payment float64
payment_type float64
water_quality float64
quality_group float64
quantity float64
quantity_group float64
source float64
source_type float64
source_class float64
waterpoint_type float64
waterpoint_type_group float64


In [14]:
predictions = predict_model(best_model, data=test)

INFO:logs:Initializing predict_model()
INFO:logs:predict_model(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=7086, verbose=0,
                       warm_start=False), probability_threshold=None, encoded_labels=False, drift_report=False, raw_score=False, round=4, verbose=True, ml_usecase=MLUsecase.CLASSIFICATION, display=None, drift_kwargs=None)
INFO:logs:Checking exceptions
INFO:logs:Preloading libraries
INFO:logs:Preparing display monitor


In [16]:
preds = predictions['Label']

In [17]:
preds

0        2
1        0
2        0
3        2
4        0
        ..
14845    2
14846    0
14847    0
14848    0
14849    2
Name: Label, Length: 14850, dtype: int64

In [18]:
arr = ['']*(len(preds))
for i in range(len(preds)):
  if preds[i] == 0:
    arr[i] = 'functional'
  elif preds[i] == 1:
    arr[i] = 'functional needs repair'
  else:
    arr[i] = 'non functional'

In [19]:
submission['status_group'] = arr

In [20]:
submission['status_group'].value_counts()

functional                 9589
non functional             4928
functional needs repair     333
Name: status_group, dtype: int64

In [21]:
submission.to_csv("6th.csv", index = False)